In [32]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans
import folium 

## Question 1: Scraping data from wiki page. Assigning it to pandas df 'toronto_data'

In [33]:
toronto_data = requests.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M")
soup = BeautifulSoup(toronto_data.content,'lxml')
table = soup.find_all('table')[0] 
df = pd.read_html(str(table))[0]  
df.head()

,Postal code,Borough,Neighborhood
0,M1A,Not assigned,NaN
1,M2A,Not assigned,NaN
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Regent Park / Harbourfront


### Formating df to exclude all rows with "Not assigned" boroughs

In [34]:
df = df[df["Borough"] != "Not assigned"].reset_index(drop=True)
df.head()

,Postal code,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Regent Park / Harbourfront
3,M6A,North York,Lawrence Manor / Lawrence Heights
4,M7A,Downtown Toronto,Queen's Park / Ontario Provincial Government


### Count rows and columns with shape.

In [35]:
df.shape

(103, 3)

## Question 2: Reading coordinate data from csv

In [36]:
coords = pd.read_csv("http://cocl.us/Geospatial_data/Geospatial_Coordinates.csv")
coords.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


### Renaming csv postal code column to match df file column name.

In [37]:
coords.rename(columns={"Postal Code": "Postal code"}, inplace=True)
coords.head()

,Postal code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


### Merging df and coords panda from csv file. 

In [40]:
df_coords = df.merge(coords, on="Postal code")
df_coords.head()

,Postal code,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,Regent Park / Harbourfront,43.654260,-79.360636
3,M6A,North York,Lawrence Manor / Lawrence Heights,43.718518,-79.464763
4,M7A,Downtown Toronto,Queen's Park / Ontario Provincial Government,43.662301,-79.389494


## Question 3: Generate Toronto map with data points

In [73]:
map_toronto = folium.Map(location=[43.651070, -79.347015], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood, postal_code in zip(df_coords['Latitude'], df_coords['Longitude'], df_coords['Borough'], df_coords['Neighborhood'], df_coords['Postal code']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='purple',
        fill=True,
        fill_color='violet',
        fill_opacity=0.5,
        parse_html=False).add_to(map_toronto)  
map_toronto

### Cluster boroughs with "Toronto" together

In [104]:
toronto_boroughs = df_coords[df_coords['Borough'].str.contains('Toronto')].reset_index(drop=True)
toronto_boroughs

,Postal code,Borough,Neighborhood,Latitude,Longitude
0,M5A,Downtown Toronto,Regent Park / Harbourfront,43.654260,-79.360636
1,M7A,Downtown Toronto,Queen's Park / Ontario Provincial Government,43.662301,-79.389494
2,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937
3,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
4,M4E,East Toronto,The Beaches,43.676357,-79.293031
5,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306
6,M5G,Downtown Toronto,Central Bay Street,43.657952,-79.387383
7,M6G,Downtown Toronto,Christie,43.669542,-79.422564
8,M5H,Downtown Toronto,Richmond / Adelaide / King,43.650571,-79.384568
9,M6H,West Toronto,Dufferin / Dovercourt Village,43.669005,-79.442259


### Visualize the 'Toronto' boroughs

In [105]:
map_toronto_boroughs = folium.Map(location=[43.651070, -79.347015], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood, postal_code in zip(toronto_boroughs['Latitude'], toronto_boroughs['Longitude'], toronto_boroughs['Borough'], toronto_boroughs['Neighborhood'], toronto_boroughs['Postal code']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='crimson',
        fill=True,
        fill_color='red',
        fill_opacity=0.5,
        parse_html=False).add_to(map_toronto_boroughs)  
map_toronto_boroughs

### Connec to Foursquare

In [149]:
CLIENT_ID = 'ID' # your Foursquare ID
CLIENT_SECRET = 'SECRET' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: ID
CLIENT_SECRET:SECRET


### Examine first neighborhood

In [109]:
toronto_boroughs.loc[0, 'Neighborhood']

'Regent Park / Harbourfront'

In [110]:
neighborhood_latitude = toronto_boroughs.loc[0, 'Latitude'] 
neighborhood_longitude = toronto_boroughs.loc[0, 'Longitude']

neighborhood_name = toronto_boroughs.loc[0, 'Neighborhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Regent Park / Harbourfront are 43.6542599, -79.3606359.


### Find top 20 venues within 500 meters of neighborhood lat/long

In [111]:
LIMIT = 20 # limit of number of venues returned by Foursquare API

radius = 500 # define radius
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
url # display URL

'https://api.foursquare.com/v2/venues/explore?&client_id=FZRTE5WPMVWLLLYJTWJYWIIUEKCPZKQPZRZR4H2BHI5NWWYP&client_secret=GI20Q4VR3PMGVOCDJ0LMO0J2DGM2MIOF2NHJX4E1G3PHBMFC&v=20180605&ll=43.6542599,-79.3606359&radius=500&limit=20'

### Process GET results for search

In [112]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5ea88d5de826ac2d0d887465'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'}]},
  'headerLocation': 'Corktown',
  'headerFullLocation': 'Corktown, Toronto',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 48,
  'suggestedBounds': {'ne': {'lat': 43.6587599045, 'lng': -79.3544279001486},
   'sw': {'lat': 43.6497598955, 'lng': -79.36684389985142}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '54ea41ad498e9a11e9e13308',
       'name': 'Roselle Desserts',
       'location': {'address': '362 King St E',
        'crossStreet': 'Trinity St',
        'lat': 43.653446723052674,
        'lng': -79.3620167174383,
        'labeledLatLngs': [{'label': 'display',
 

### Borrow get_category_type from Foursquare

In [113]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

### Structure json into dataframe

In [117]:
import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues

,name,categories,lat,lng
0,Roselle Desserts,Bakery,43.653447,-79.362017
1,Tandem Coffee,Coffee Shop,43.653559,-79.361809
2,Morning Glory Cafe,Breakfast Spot,43.653947,-79.361149
3,Cooper Koo Family YMCA,Distribution Center,43.653249,-79.358008
4,Body Blitz Spa East,Spa,43.654735,-79.359874
5,Impact Kitchen,Restaurant,43.656369,-79.356980
6,Corktown Common,Park,43.655618,-79.356211
7,The Extension Room,Gym / Fitness Center,43.653313,-79.359725
8,The Distillery Historic District,Historic Site,43.650244,-79.359323
9,Figs Breakfast & Lunch,Breakfast Spot,43.655675,-79.364503


In [116]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

20 venues were returned by Foursquare.


### Repeating process for all "Toronto" Neighborhoods

In [118]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [119]:
toronto_venues = getNearbyVenues(names=df_coords['Neighborhood'],
                                   latitudes=df_coords['Latitude'],
                                   longitudes=df_coords['Longitude']
                                  )


Parkwoods
Victoria Village
Regent Park / Harbourfront
Lawrence Manor / Lawrence Heights
Queen's Park / Ontario Provincial Government
Islington Avenue
Malvern / Rouge
Don Mills
Parkview Hill / Woodbine Gardens
Garden District, Ryerson
Glencairn
West Deane Park / Princess Gardens / Martin Grove / Islington / Cloverdale
Rouge Hill / Port Union / Highland Creek
Don Mills
Woodbine Heights
St. James Town
Humewood-Cedarvale
Eringate / Bloordale Gardens / Old Burnhamthorpe / Markland Wood
Guildwood / Morningside / West Hill
The Beaches
Berczy Park
Caledonia-Fairbanks
Woburn
Leaside
Central Bay Street
Christie
Cedarbrae
Hillcrest Village
Bathurst Manor / Wilson Heights / Downsview North
Thorncliffe Park
Richmond / Adelaide / King
Dufferin / Dovercourt Village
Scarborough Village
Fairview / Henry Farm / Oriole
Northwood Park / York University
East Toronto
Harbourfront East / Union Station / Toronto Islands
Little Portugal / Trinity
Kennedy Park / Ionview / East Birchmount Park
Bayview Village
Do

In [120]:
print(toronto_venues.shape)
toronto_venues.head()

(1076, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Parkwoods,43.753259,-79.329656,Brookbanks Park,43.751976,-79.332140,Park
1,Parkwoods,43.753259,-79.329656,Brookbanks Pool,43.751389,-79.332184,Pool
2,Parkwoods,43.753259,-79.329656,Variety Store,43.751974,-79.333114,Food & Drink Shop
3,Victoria Village,43.725882,-79.315572,Victoria Village Arena,43.723481,-79.315635,Hockey Arena
4,Victoria Village,43.725882,-79.315572,Tim Hortons,43.725517,-79.313103,Coffee Shop


### Run venue count for neighborhoods

In [122]:
toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Agincourt,4,4,4,4,4,4
Alderwood / Long Branch,9,9,9,9,9,9
Bathurst Manor / Wilson Heights / Downsview North,20,20,20,20,20,20
Bayview Village,4,4,4,4,4,4
Bedford Park / Lawrence Manor East,20,20,20,20,20,20
Berczy Park,20,20,20,20,20,20
Birch Cliff / Cliffside West,4,4,4,4,4,4
Brockton / Parkdale Village / Exhibition Place,20,20,20,20,20,20
Business reply mail Processing CentrE,18,18,18,18,18,18


### Find unique categories

In [123]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 215 uniques categories.


### Analyze each neighborhood

In [124]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Yoga Studio,Accessories Store,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Art Gallery,...,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wine Shop,Wings Joint,Women's Store
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [125]:
toronto_onehot.shape

(1076, 215)

In [126]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped

,Neighborhood,Yoga Studio,Accessories Store,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,...,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wine Shop,Wings Joint,Women's Store
0,Agincourt,0.000000,0.0,0.000000,0.0000,0.0000,0.000,0.0000,0.0000,0.000000,...,0.00,0.00,0.00,0.00,0.0000,0.000000,0.0,0.000000,0.0,0.00
1,Alderwood / Long Branch,0.000000,0.0,0.000000,0.0000,0.0000,0.000,0.0000,0.0000,0.000000,...,0.00,0.00,0.00,0.00,0.0000,0.000000,0.0,0.000000,0.0,0.00
2,Bathurst Manor / Wilson Heights / Downsview North,0.000000,0.0,0.000000,0.0000,0.0000,0.000,0.0000,0.0000,0.000000,...,0.00,0.00,0.00,0.05,0.0000,0.000000,0.0,0.000000,0.0,0.00
3,Bayview Village,0.000000,0.0,0.000000,0.0000,0.0000,0.000,0.0000,0.0000,0.000000,...,0.00,0.00,0.00,0.00,0.0000,0.000000,0.0,0.000000,0.0,0.00
4,Bedford Park / Lawrence Manor East,0.000000,0.0,0.000000,0.0000,0.0000,0.000,0.0000,0.0000,0.050000,...,0.00,0.00,0.00,0.00,0.0000,0.000000,0.0,0.000000,0.0,0.00
5,Berczy Park,0.000000,0.0,0.000000,0.0000,0.0000,0.000,0.0000,0.0000,0.000000,...,0.00,0.00,0.05,0.00,0.0000,0.000000,0.0,0.000000,0.0,0.00
6,Birch Cliff / Cliffside West,0.000000,0.0,0.000000,0.0000,0.0000,0.000,0.0000,0.0000,0.000000,...,0.00,0.00,0.00,0.00,0.0000,0.000000,0.0,0.000000,0.0,0.00
7,Brockton / Parkdale Village / Exhibition Place,0.000000,0.0,0.000000,0.0000,0.0000,0.000,0.0000,0.0000,0.000000,...,0.00,0.00,0.00,0.00,0.0000,0.000000,0.0,0.000000,0.0,0.00
8,Business reply mail Processing CentrE,0.000000,0.0,0.000000,0.0000,0.0000,0.000,0.0000,0.0000,0.000000,...,0.00,0.00,0.00,0.00,0.0000,0.000000,0.0,0.000000,0.0,0.00
9,CN Tower / King and Spadina / Railway Lands / ...,0.000000,0.0,0.062500,0.0625,0.0625,0.125,0.1875,0.0625,0.000000,...,0.00,0.00,0.00,0.00,0.0000,0.000000,0.0,0.000000,0.0,0.00


In [127]:
toronto_grouped.shape

(94, 215)

### Print each neighborhood with the top 5 venues

In [128]:
num_top_venues = 5

for hood in toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Agincourt----
                       venue  freq
0                     Lounge  0.25
1  Latin American Restaurant  0.25
2             Breakfast Spot  0.25
3               Skating Rink  0.25
4                      Motel  0.00


----Alderwood / Long Branch----
          venue  freq
0   Pizza Place  0.22
1   Coffee Shop  0.11
2  Skating Rink  0.11
3           Gym  0.11
4          Pool  0.11


----Bathurst Manor / Wilson Heights / Downsview North----
                       venue  freq
0                       Bank  0.10
1                Coffee Shop  0.10
2             Sandwich Place  0.05
3  Middle Eastern Restaurant  0.05
4                 Restaurant  0.05


----Bayview Village----
                 venue  freq
0   Chinese Restaurant  0.25
1                 Bank  0.25
2                 Café  0.25
3  Japanese Restaurant  0.25
4          Yoga Studio  0.00


----Bedford Park / Lawrence Manor East----
                venue  freq
0      Sandwich Place  0.10
1  Italian Restaurant  0.10
2      

### Sort venues descending order

In [129]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

### Display top 10 venues for each neighborhood

In [133]:
import numpy as np
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Agincourt,Latin American Restaurant,Lounge,Skating Rink,Breakfast Spot,Department Store,Drugstore,Donut Shop,Dog Run,Distribution Center,Discount Store
1,Alderwood / Long Branch,Pizza Place,Pool,Coffee Shop,Pharmacy,Gym,Skating Rink,Sandwich Place,Pub,Women's Store,Deli / Bodega
2,Bathurst Manor / Wilson Heights / Downsview North,Coffee Shop,Bank,Fried Chicken Joint,Ice Cream Shop,Supermarket,Shopping Mall,Middle Eastern Restaurant,Deli / Bodega,Pizza Place,Restaurant
3,Bayview Village,Chinese Restaurant,Bank,Japanese Restaurant,Café,Women's Store,Department Store,Eastern European Restaurant,Drugstore,Donut Shop,Dog Run
4,Bedford Park / Lawrence Manor East,Coffee Shop,Sandwich Place,Italian Restaurant,Thai Restaurant,Comfort Food Restaurant,Pharmacy,Café,Butcher,Pub,Restaurant


### Run k-means to cluster neighborhoods into 5 clusters

In [134]:
# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0], dtype=int32)

### Create new df with clusters

In [135]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = df_coords

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

toronto_merged.head() # check the last columns!

,Postal code,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M3A,North York,Parkwoods,43.753259,-79.329656,1.0,Park,Pool,Food & Drink Shop,Women's Store,Deli / Bodega,Drugstore,Donut Shop,Dog Run,Distribution Center,Discount Store
1,M4A,North York,Victoria Village,43.725882,-79.315572,0.0,Coffee Shop,French Restaurant,Portuguese Restaurant,Hockey Arena,Eastern European Restaurant,Drugstore,Donut Shop,Dog Run,Distribution Center,Discount Store
2,M5A,Downtown Toronto,Regent Park / Harbourfront,43.654260,-79.360636,0.0,Coffee Shop,Park,Breakfast Spot,Bakery,Performing Arts Venue,Chocolate Shop,Pub,Mexican Restaurant,Restaurant,Dessert Shop
3,M6A,North York,Lawrence Manor / Lawrence Heights,43.718518,-79.464763,0.0,Furniture / Home Store,Clothing Store,Women's Store,Boutique,Miscellaneous Shop,Arts & Crafts Store,Event Space,Coffee Shop,Accessories Store,Vietnamese Restaurant
4,M7A,Downtown Toronto,Queen's Park / Ontario Provincial Government,43.662301,-79.389494,0.0,Coffee Shop,Sushi Restaurant,Yoga Studio,Theater,Creperie,Mexican Restaurant,Burrito Place,Diner,Distribution Center,Beer Bar


### Visualize cluster data

In [148]:
# create map
map_clusters = folium.Map(location=[43.651070, -79.347015], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[-1],
        fill=True,
        fill_color=[-1],
        fill_opacity=0.5).add_to(map_clusters)
       
map_clusters